# Feature Extraction, NLP and Naive-Bayes: OKCupid Profiles

## Introduction:
This project analyzes data from the online dating application, OKCupid, that is a summary of profile information from users in the San Francisco Bay Area. The goal is to scope, prep, analyze/extract features and create a machine learning model to answer a question:
Can you predict if someone works in a STEM (Science, Technology, Engineering and Math) profession based on their user profile data?

**Data sources:**

`profiles.csv` was provided by Codecademy.com with permission from OKCupid.

## Scope:
### Project Goal
The goal is to use exploratory data analysis to identify potential features of OKCupid profiles that may help predict whether or not an individual works in the STEM field. I will then use these features to build and evaluate a Naive-Bayes model. This will demonstrate and exercise skills in data cleaning, feature extraction, natural language processing and model evaluation.
### Data
The project has one data set provided by Codecademy called profiles.csv. In the data, each row represents an OkCupid user and the columns are the responses to their user profile questions, which include multiple-choice and short answer questions. It consists of a mixture of numerical, categorical and free-text variables. I will be focusing on the free-text variables for this example.
### Analysis
I will use natural language processing to find key terms that will most likely provide significant predictive power for a Naive-Bayes model. 
### Evaluation
Model performance will be initially evaluated based on Area Under the Receiver Operating Characteristic Curve (ROC AUC). The final model will also be evaluated based on accuracy score and confusion matrix statistics with a discussion on how to select a threshold based on profit-matrix values.

## Load General Libraries and Data:

In [1]:
## General libraries
import numpy as np
import pandas as pd

# Visualization libraries
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# load profile data
profiles = pd.read_csv('profiles.csv', encoding='utf-8')

profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   body_type    54650 non-null  object 
 2   diet         35551 non-null  object 
 3   drinks       56961 non-null  object 
 4   drugs        45866 non-null  object 
 5   education    53318 non-null  object 
 6   essay0       54458 non-null  object 
 7   essay1       52374 non-null  object 
 8   essay2       50308 non-null  object 
 9   essay3       48470 non-null  object 
 10  essay4       49409 non-null  object 
 11  essay5       49096 non-null  object 
 12  essay6       46175 non-null  object 
 13  essay7       47495 non-null  object 
 14  essay8       40721 non-null  object 
 15  essay9       47343 non-null  object 
 16  ethnicity    54266 non-null  object 
 17  height       59943 non-null  float64
 18  income       59946 non-null  int64  
 19  job 

In [3]:
# create new dataframe based on columns associated with free-text or the target variable
profiles_text = profiles[['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 
                          'essay6', 'essay7', 'essay8', 'essay9', 'job']]

# helper method to quantify percentage of missingness
def percent_missing(df):
    print((100*df.isna().sum().sort_values(ascending=False)/len(df)).round(2))
    
# show missing percents    
percent_missing(profiles_text)
profiles_text.info()


essay8    32.07
essay6    22.97
essay9    21.02
essay7    20.77
essay3    19.14
essay5    18.10
essay4    17.58
essay2    16.08
job       13.68
essay1    12.63
essay0     9.15
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   essay0  54458 non-null  object
 1   essay1  52374 non-null  object
 2   essay2  50308 non-null  object
 3   essay3  48470 non-null  object
 4   essay4  49409 non-null  object
 5   essay5  49096 non-null  object
 6   essay6  46175 non-null  object
 7   essay7  47495 non-null  object
 8   essay8  40721 non-null  object
 9   essay9  47343 non-null  object
 10  job     51748 non-null  object
dtypes: object(11)
memory usage: 5.0+ MB


## Data Summary:
Profiles_text consists of 11 columns and 59946 rows. Column content and missingness are discussed below.

**The columns in the dataset include:**
- job: nominal variable of employment description (will convert to binary target, STEM = 1 and Other = 0)

**And a set of open short-answer responses to :**
- essay0: My self summary
- essay1: What I’m doing with my life
- essay2: I’m really good at
- essay3: The first thing people usually notice about me
- essay4: Favorite books, movies, show, music, and food
- essay5: The six things I could never do without
- essay6: I spend a lot of time thinking about
- essay7: On a typical Friday night I am
- essay8: The most private thing I am willing to admit
- essay9: You should message me if…

**Percent of Missingness by Column:**
    
     essay8      32.07
     essay6      22.97
     essay9      21.02
     essay7      20.77
     essay3      19.14
     essay5      18.10
     essay4      17.58
     essay2      16.08
     job         13.68
     essay1      12.63
     essay0      9.15
     
     
## Data Exploration, Transformation and Feature Extraction:
### Creating the Target
All rows with a null value for 'essay0' or 'job' were dropped leaving 47,489 total. A new column was created to represent the target variable for STEM jobs. Any users that answered 'science / tech / engineering', 'computer / hardware / software', or 'medicine / health' were considered to work in the STEM field. The target class represents ~25.6% of the sample, which means it is moderately imbalanced. If I downsample the non-STEM examples, there will be a loss of data about non-STEM users that could potentially hurt the model performance. Conversely, oversampling the minority can lead to overfitting and poor future performance of the model. During model training, various techniques for oversampling the miniority class (SMOTE, BorderlineSMOTE, SMOTEN) and undersampling (RandomUnderSampling) the majority class will be empirically evaluated.

In [4]:
# drop any rows with null values for essay0, which will guarantee some free-text data for each row
profiles_text = profiles_text[profiles_text['essay0'].notna()].reset_index()

# drop any rows with null values for job
profiles_text = profiles_text[profiles_text['job'].notna()].reset_index()

# check new length of dataframe (OUTPUT: 51748)
print(len(profiles_text))

# create a list of values for STEM field jobs
STEM_list = ['science / tech / engineering', 'computer / hardware / software', 'medicine / health']

# create new column 'STEM' to hold binary target variable. 1 = STEM field job
profiles_text['STEM'] = profiles_text.apply(lambda row: 1 if row['job'] in STEM_list else 0, axis=1)

# check for target class balance
print(profiles_text.STEM.sum()/len(profiles_text))
# target is about 25.6%, which is moderately imbalanced. Consider resampling STEM jobs or downsampling non-STEM

47489
0.2567120806923709


## Cleaning and Prepping for Analysis

## Natural Language Processing:
### Preprocessing Text
1. Import libraries from nltk, the Natural Language Toolkit.
2. Collect document column names into a list.
3. Replace null documents with empty strings.
4. Convert text to lowercase and remove punctuation. 
5. Combine all text columns into one column, the corpus with each row representing a document.

At this point the number of unique words in the corpus was exceptionally large, 148,517 as seen with value_counts(). The more common a word in the corpus, the less predictive power it will have. Also, if it is too rare, the word will not be prevalent enough to train the model. A threshold of at least 50 occurrences in the corpus was chosen as a minimum. Additionally, the 100 most common words were eliminated as stop words. This reduced the corpus vocabulary to 13,507 words, which is much more manageable. To save on processing time, a whitelist of the 13,507 word vocabulary was used. A stop_word list was ten times as long and would have taken roughly ten times the processing time. 

6. Remove common and rare words.
7. Split document into word tokens.
8. Lemmatize word tokens by converting words to their common root. Most accurate when done with the words part of speech. An example: 'are', 'is' and 'will' become 'be' after lemmatization. This helps reduce the overall vocabulary size.

**Note:** Steps 4 and 6-8 could have been handled by CountVectorizer() with default/custom settings for stop_words and lemmatization with a smaller collection of data. With one this size, I received a memory usage error.


In [5]:
# load libraries for NLP
import nltk, re
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

# create list of essay columns
essays = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4',\
          'essay5', 'essay6', 'essay7', 'essay8', 'essay9']

# fill in null essays with empty strings
profiles_text[essays] = profiles_text[essays].replace(np.nan, '', regex= True)

# convert all text to lowercase, remove punctuation
for essay in essays:
    profiles_text[essay] = profiles_text[essay].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    profiles_text[essay] = profiles_text[essay].str.replace('[^\w\s]',' ', regex=True)

# combine all essay column text in a new column 'all_essays'
profiles_text['all_essays'] = profiles_text[essays].apply(lambda x: ' '.join(x), axis= 1)

# find somewhat common words (occurence rate of >50, but not the 100 most common in corpus)
white_list_words = pd.Series(' '.join(profiles_text['all_essays'])\
                                .split()).value_counts()[100:]\
                                .loc[lambda x: x > 50].index.tolist()

# this step is resource intensive. checks each word in all_essays against the whitelist of 13507.
# a stopword list with the 100 most frequent and the words with term frequencies < 50 was 135010 terms long.
# in this case, checking against a white list is 10x faster
profiles_text['all_essays'] = profiles_text['all_essays'].apply(lambda x: ' '.join(x for x in x.split() if x in white_list_words))

# create a lemmatizer object
normalizer = WordNetLemmatizer()

# helper method to retrieve most likely part of speech for a word
def get_part_of_speech(word):
    # retrieve parts of speech from wordnet.synsets()
    probable_part_of_speech = wordnet.synsets(word)
    
    # create counter object
    pos_counts = Counter()
    
    # count parts of speech
    pos_counts['n'] = len([item for item in probable_part_of_speech if item.pos()=='n'])
    pos_counts['v'] = len([item for item in probable_part_of_speech if item.pos()=='v'])
    pos_counts['a'] = len([item for item in probable_part_of_speech if item.pos()=='a'])
    pos_counts['r'] = len([item for item in probable_part_of_speech if item.pos()=='r'])
    
    # assign part of speech to the most common
    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    
    return most_likely_part_of_speech

# helper method to preprocess text and return a lemmatized list of words
def preprocess_text(text):
    # remove non-word characters and spaces, convert to lower-case
    cleaned = re.sub(r'\W+', ' ', text).lower()
    # create a tokenized list of words
    tokenized = word_tokenize(text)
    # lemmatize tokenized list with the part of speech fed to the lemmatizer
    normalized = " ".join([normalizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized])
    
    return normalized

# tokenize and lemmatize the words in all_essays
profiles_text['all_essays'] = profiles_text.apply(lambda row: preprocess_text(row['all_essays']), axis= 1)


#### Extracting Keywords by Odds-Ratios Adjusted for False Discovery Rate Between STEM and nonSTEM Profiles

The goal of this processing is to extract keywords that are particularly associated with one class over another using an odds-ratio. If the words are equally associated with each class the ratio is one. The higher above one, the more closely it is associated with STEM words. The closer the ratio gets to 0, the more it is associated with nonSTEM profiles.

1. The profiles are divided based on the target variable.
2. The CountVectorizer is used in binary mode to return if a vocabulary word is found in a user's combined essays document.
3. This information is stored in a dataframe by each class.
4. The binary row for each word is summed for the class. This equates to the number of profiles where that word occurs.
5. The number from step 4 is subtracted from the total number of class profiles. This is the number of profiles that did not have that word.
6. The dataframes for each class are combined using an inner join on the indexes (words).
7. Between class odds-ratios and p-values for each word are computed with Fisher's exact test method from Scipy.stats.
8. Because this is a test run thousands of times, there is a large risk of false-positives accumulating. To counteract a high false-positive rate, Benjamani-Hochberg correction for p-values was applied.
9. A cutoff of a corrected p-value less than 1.0 x 10^-6 and an odds ratio greater than 2 or less than 0.5 was chosen to limit the keyword selection.

**Interpretation:** Words where the interclass odds-ratio is greater than 2 indicate these words are at least twice as likely to occur in a STEM profile. Words where the interclass odds-ratio is less than 0.5 are twice as likely to occur in a non-STEM profile. A corrected p-value less than 1.0 x 10^-6 means that the probability of these odds-ratios occuring by chance alone are less than one in a million.

**This 2x odds-ratio cutoff yielded 120 terms found in the lists below:**

    104 STEM words:
     
    '_blank',        'algorithm',      'alto',           'anathem',         'android',  
    'application',   'apps',           'artificial',     'asimov',          'bicycling', 
    'biomedical',    'biotech',        'bt',             'canada',          'clinic', 
    'clinical',      'cod',            'code',           'commute',         'computer', 
    'computing',     'coworkers',      'crossfit',       'cryptonomicon',   'data', 
    'database',      'developer',      'dna',            'electrical',      'electronics', 
    'emergency',     'ender',          'engineer',       'engineering',     'escher', 
    'feynman',       'fm',             'freakonomics',   'gadget',          'gattaca', 
    'geeky',         'gibson',         'hack',           'hacker',          'hardware', 
    'healthcare',    'heinlein',       'hospital',       'iain',            'intp',            
    'javascript',    'lab',            'laser',          'linux',           'mechanical',      
    'medical',       'mit',            'mobile',         'molecular',       'neal',            
    'neuromancer',   'neuroscience',   'nofollow',       'nurse',           'orbital',         
    'palo',          'pediatric',      'pharmaceutical', 'physician',       'practitioner',    
    'pratchett',     'primer',         'programmer',     'programming',     'psychotherapist', 
    'register',      'rel',            'researcher',     'residency',       'rn',             
    'scientific',    'scientist',      'scifi',          'silicon',         'software',        
    'solar',         'soma',           'startup',        'stephenson',      'stross',       
    'tech',          'techie',         'technical',      'technician',      'technology',     
    'therapist',     'tinker',         'ucsf',           'ui',              'vernor',        
    'vinge',         'web',            'wikipedia',      'xkcd'
        
It appears to be a success. STEM words are associated with STEM professions, like 'rn' for Registered Nurse or 'developer'. They also play to popculture stereotypes. For example, STEM users talk about Neal Stephenson's books "Anathem" and "Cryptonomicon" and Sci-Fi authors Vernor Vinge and Charles Stross. They also mention noted physicist Richard Feynman and my personal favorite science themed webcomic, 'XKCD'. For exercise, they seem to enjoy 'bicycling' and 'crossfit'. 

    16 non-STEM words:
     
    'accounting',    'attorney',       'ba',           'credential', 
    'educator',      'elementary',     'homework',     'lawyer', 
    'mfa',           'nanny',          'retail',       'semester', 
    'sfsu',          'student',        'stylist',      'teacher'
    


Many nonSTEM profiles refer to nonSTEM professions, like 'attorney', 'accounting', 'nanny' and 'hairstylist'. They also refer to Arts education majors like 'BA' and 'MFA'. They also might still be a 'student' at 'SFSU'. 

Keywords associated with pop-culture will shift in prevalence with the changing popularity of topics in the populace. So while they are a good indicator for this data and time period, they will need to be reupdated if the model is to be used over time. The profession associated words will likely remain a more stable predictor in the long run.  

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# create subset of only STEM profiles and a document collection of only STEM profile essays
STEM_data = profiles_text[profiles_text['STEM'] == 1]
STEM_text_data = STEM_data['all_essays']

# store number of STEM profiles for future calculations
num_STEM = len(STEM_data)
# create column headers for a dataframe to hold STEM profile binary-word counts. P=positive target class.
STEM_profiles = [f"P {i+1}" for i in range(num_STEM)]

# create a CountVectorizer object in binary mode. Fit_transform on STEM_text_data
vectorizer = CountVectorizer(binary=True)
STEM_vectors = vectorizer.fit_transform(STEM_text_data)

# get the feature_names to act as row indexes in STEM_counts dataframe
feature_names = vectorizer.get_feature_names()

# transform the sparse matrix result from the CountVectorizer into a dataframe. 
# Rows= words, Columns= each profile, Values= binary for each term
STEM_counts = pd.DataFrame(STEM_vectors.T.todense(), index=feature_names, columns=STEM_profiles) 

In [7]:
# create STEM_sum_pos column that is the number of STEM profiles that used a word
STEM_counts['STEM_sum_pos'] = STEM_counts.apply(lambda x: np.sum(x[STEM_profiles]), axis=1)

# create STEM_sum_neg column that is the number of STEM profiles that did not use a word
STEM_counts['STEM_sum_neg'] = num_STEM - STEM_counts['STEM_sum_pos']

# create nonSTEM subset and document collection
nonSTEM_data = profiles_text[profiles_text['STEM'] == 0]
nonSTEM_text_data = nonSTEM_data['all_essays']

# store number of nonSTEM profiles for future calculations
num_nonSTEM = len(nonSTEM_data)

# create column headers. N= negative target class
nonSTEM_profiles = [f"N {i+1}" for i in range(num_nonSTEM)]

# transform nonSTEM_text_data with previously fit CountVectorizer object
nonSTEM_vectors = vectorizer.transform(nonSTEM_text_data)

# create nonSTEM dataframe from previous steps sparse matrix results. Same structure as STEM_counts
nonSTEM_counts = pd.DataFrame(nonSTEM_vectors.T.todense(), index= feature_names, columns= nonSTEM_profiles) 

# create nonSTEM_sum_pos column
nonSTEM_counts['nonSTEM_sum_pos'] = nonSTEM_counts.apply(lambda x: np.sum(x[nonSTEM_profiles]), axis=1)

# create nonSTEM_sum_neg column
nonSTEM_counts['nonSTEM_sum_neg'] = num_nonSTEM - nonSTEM_counts['nonSTEM_sum_pos']

# combine class dataframes into one using an innerjoin on indexes
all_counts = STEM_counts.merge(nonSTEM_counts, left_index=True, right_index=True)


In [8]:
# import fisher_exact for computing odds-ratios and p-values
from scipy.stats import fisher_exact
# import multipletests to calculate adjusted p-values
from statsmodels.stats.multitest import multipletests

# helper method to run fisher's exact test
def odds_ratio(class1_pos, class2_pos, class1_neg, class2_neg):
    odds_ratio, p_value = fisher_exact([[class1_pos, class2_pos],
                                        [class1_neg, class2_neg]])
    return odds_ratio, p_value

# store odds_ratio in column
all_counts['odds_ratio'] = all_counts.apply(lambda x: odds_ratio(x['STEM_sum_pos'], x['nonSTEM_sum_pos'],
                                                                 x['STEM_sum_neg'], x['nonSTEM_sum_neg'])[0], axis=1)
# store unadjusted p-values in column
all_counts['odds_p_value'] = all_counts.apply(lambda x: odds_ratio(x['STEM_sum_pos'], x['nonSTEM_sum_pos'],
                                                                 x['STEM_sum_neg'], x['nonSTEM_sum_neg'])[1], axis=1)

In [9]:
# helper method to calculate and return adjusted p-value
def corrected_pval(p_values):
    rej, pvals, aSidak, aBonf = multipletests(p_values, alpha=.05, method='fdr_bh')
    
    return pvals

# store corrected p-value in new column
all_counts['FDR_corrected_p_value'] = corrected_pval(all_counts['odds_p_value'])

In [10]:
# store keywords in two lists based on class affinity.
STEM_words = all_counts[(all_counts['FDR_corrected_p_value'] < 0.000001) & (all_counts['odds_ratio'] > 2)].index.tolist()
nonSTEM_words = all_counts[(all_counts['FDR_corrected_p_value'] < 0.000001) & (all_counts['odds_ratio'] < .5)].index.tolist()

# print length of the lists and the keywords associated with each
print(len(STEM_words), STEM_words)
print(len(nonSTEM_words), nonSTEM_words)

104 ['_blank', 'algorithm', 'alto', 'anathem', 'android', 'application', 'apps', 'artificial', 'asimov', 'bicycling', 'biotech', 'bt', 'canada', 'clinic', 'clinical', 'cod', 'code', 'commute', 'computer', 'computing', 'coworkers', 'crossfit', 'cryptonomicon', 'data', 'database', 'developer', 'dna', 'electrical', 'electronics', 'emergency', 'engineer', 'engineering', 'escher', 'feynman', 'fm', 'freakonomics', 'gadget', 'gattaca', 'geeky', 'gibson', 'hack', 'hacker', 'hardware', 'healthcare', 'heinlein', 'hospital', 'iain', 'intp', 'javascript', 'lab', 'laser', 'linux', 'mechanical', 'medical', 'mit', 'mobile', 'molecular', 'neal', 'neuromancer', 'neuroscience', 'nofollow', 'nurse', 'orbital', 'palo', 'paramedic', 'pediatric', 'pharmaceutical', 'physician', 'practitioner', 'pratchett', 'primer', 'programmer', 'programming', 'psychotherapist', 'register', 'rel', 'researcher', 'residency', 'rn', 'scientific', 'scientist', 'scifi', 'silicon', 'singularity', 'software', 'solar', 'solve', 'so

### Create Binary Features for Keywords

In [11]:
#combine lists for keywords
keywords = STEM_words + nonSTEM_words

# helper method to check if a word is in the profiles essays
def get_keyword_binary(keyword, essay_str):
    if keyword in essay_str:
        return 1
    else:
        return 0
    
# loop through keyword list and create a new binary column of that word in essays
for keyword in keywords:
    profiles_text[keyword] = profiles_text.apply(lambda row: get_keyword_binary(keyword, row['all_essays']), axis=1)

## Modeling Plan:

The data will be split into training (0.8) and test set (0.2) portions. Using 5-fold cross-validation, keyword features will be run through a Naive-Bayes model. ROC-AUC scores will be computed for each k-fold and the overall average ROC-AUC score will be used for evaluation. ROC-AUC is used because it is a good indicator of overall model performance, independent of the threshold used for classification. Accuracy is not the best statistic because it would be heavily affected by the threshold. 

Finally, the resultant best model will be scored on the test data to see how training with cross-validation for the model might perform with new samples, indicating future performance.

**Note:** Not rebalancing the classes and various methods of oversampling the minority class, undersampling the majority class or a combination of over/undersampling were tried. For oversampling, variations of Synthetic Minority Oversampling Technique (SMOTE) were used from the imbalanced-learning (imblearn) library. For undersampling, Random Under Sampling was used. Based on a comparison of ROC-AUC scores in the test data, not resampling performed the best empirically and was chosen for model fitting. It was also the only method to perform with a ROC-AUC over 0.67 with the test data.


### Modeling
1. Split data into training and test sets.
2. Create Naive_Bayes model.
3. Split training sets into 5 k-folds for cross validation.
4. Evaluate each model with training data based on ROC-AUC.
5. Fit on best trial model and run associated test data.
6. Evaluate final model performance.

In [12]:
# import tools for modeling
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_recall_fscore_support

# split into X data and y labels
X = profiles_text[keywords]
y = profiles_text['STEM']


# train test split with 0.2 test portion. random_state set to allow for tuning comparisons.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# create ComplementNB() model. Using this Naive_Bayes model because it is suited to imbalanced class problems.
cnb = ComplementNB()

# create 5 k-folds for cross-validation. random_state set to keep consistency between trials for model type and tuning.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= 42)

# run cross-validation and store results 
cv_results = cross_val_score(cnb,
                             X_train,
                             y_train,
                             cv= skf,
                             scoring = 'roc_auc')

# print each result with the average for all trials
print(cv_results, np.mean(cv_results))


[0.73943844 0.73439825 0.71608594 0.73536559 0.73587263] 0.7322321704951167


In [13]:
# fit model and store predictions
predictions = cnb.fit(X_train, y_train)

# run model on test data
predictions = cnb.predict(X_test)

# print summary of results
print(roc_auc_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(accuracy_score(y_test, predictions))
print(precision_recall_fscore_support(y_test, predictions))

0.6766157947467512
[[5749 1326]
 [1113 1310]]
0.7432090966519267
(array([0.83780239, 0.4969651 ]), array([0.81257951, 0.54065208]), array([0.82499821, 0.51788891]), array([7075, 2423], dtype=int64))


## Results: 

**Training Data:** 

- ROC-AUC Cross Validation = 0.73943844, 0.73439825, 0.71608594, 0.73536559, 0.73587263
- ROC-AUC Average = 0.7322321704951167

**Test Data:** 

- ROC-AUC = 0.6766157947467512
- Accuracy = 0.7432090966519267
- Confusion Matrix = 
                        
                              PREDICTED
                        | nonSTEM | STEM |
       ACTUAL | nonSTEM |    5749 | 1326 |
              |    STEM |    1113 | 1310 |


- STEM precision = 0.4969651 
- STEM recall = 0.54065208
- STEM F1 = 0.51788891
- Test data num_STEM (support) = 2423

- nonSTEM precision = 0.83780239
- nonSTEM recall = 0.81257951
- nonSTEM F1 = 0.82499821
- Test data num_nonSTEM (support) = 7075

## Interpretation:

In general, ROC-AUC scores of 0.7-0.8 are considered acceptable. Performance on the test data was just under this threshold. For context, a score of 0.5 is indicative of random guessing for equally distributed classes and a score of 1 is a perfect model. The cross-validation model performance with only the keyword data features indicates the value of information that can be derived from free-text sources.

Looking at the test data confusion matrix and by class statistics, this model does not necessarily predict STEM profiles well, but performs well in predicting non-STEM profiles. Overall, the errors are most likely due to the amount of missing information in the original data and the fact that the test data classes were not rebalanced. In this case the model derives its prediction from the free-text data, and many profiles did not fill out all of the essays. As such, when computing probability of class, the model has limited information and was more often correct in the test data when assigning non-STEM due to a class imbalance in the population. A random guess between classes is more likely to be correct if it is non-STEM because the population occurrence of non-STEM profiles is roughly 3:1. The frequency of error types could be adjusted by lowering or raising the threshold of prediction for STEM profiles. Normally this would be done with the use of a profit-matrix where a different penalty (increased risk or financial cost) is assigned to the two types of error, false-positives and false-negatives. Based on the cost of each type of error, the decision threshold would be adjusted to shift the balance of the errors towards the less costly type.

## Conclusion:

This was a good exercise in keyword feature extraction with natural language processing and strategies for dealing with imbalanced target classes. Similar NLP techniques can be used for summarizing or classifying documents, emails, online reviews and free-form responses in surveys. Typically tf-idf vectors from the count-vectorizer are fed into a model, but are difficult to interpret exactly how the model is arriving at its decision. Extracting the odds-ratio and identifying the keywords that are being used allows insight into the method of prediction and can avoid unintentional bias in the real world. 
